In [2]:
import numpy as np

data = np.load('A07T.npz')  
signal = data['s']  # Assuming shape is (total_samples, 22)
event_types = data['etyp'].T[0]
event_positions = data['epos'].T[0]

# Initialize empty arrays
signals = []
trial_types = []
valid_labels = {769, 770, 771, 772}

for i in range(0, len(event_positions) - 1):
    event_type = event_types[i]
    next_event_type = event_types[i + 1]
    
    if event_type == 768 and next_event_type in valid_labels:  # Valid trial start
        pos = event_positions[i+1]
        
        # Extract 750 samples x 22 channels
        trial_signal = signal[pos+750 : pos+1500, 0:22]  # All 22 channels
        
        # Verify the shape is correct
        if trial_signal.shape != (750, 22):
            print(f"Unexpected shape at trial {len(signals)}: {trial_signal.shape}")
            continue
            
        signals.append(trial_signal)
        trial_types.append(next_event_type)

# Convert to numpy arrays
signals_array = np.array(signals)  # Shape (288, 750, 22)
labels_array = np.array(trial_types)  # Shape (288,)

# Verify final shapes
print("Signals shape:", signals_array.shape)
print("Labels shape:", labels_array.shape)
print("Unique labels:", np.unique(labels_array))

Unexpected shape at trial 270: (727, 22)
Signals shape: (270, 750, 22)
Labels shape: (270,)
Unique labels: [769 770 771 772]


In [3]:
np.save('eeg_signals.npy', signals_array)
np.save('eeg_labels.npy', labels_array)

In [4]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch.nn as nn 
import torch.optim as optim 

class EEGDataset(Dataset):
    def __init__(self, trials, labels):
        # Trials: (num_trials, 750, 22)
        # Labels: (num_trials,)
        
        # Normalize data per channel
        self.data = torch.tensor(trials, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)
        
        # Map original labels to 0-3
        self.label_mapping = {769: 0, 770: 1, 771: 2, 772: 3}
        self.labels = torch.tensor([self.label_mapping[int(x)] for x in labels])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

# Assuming you have loaded your data into trials and labels arrays
trials = np.load('eeg_signals.npy')  # Shape (288, 750, 22)
labels = np.load('eeg_labels.npy')     # Shape (288,)

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    trials, labels, test_size=0.2, stratify=labels, random_state=42
)

# Create datasets and dataloaders
train_dataset = EEGDataset(X_train, y_train)
test_dataset = EEGDataset(X_test, y_test)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class LTC_Cell(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(LTC_Cell, self).__init__()
        self.hidden_dim = hidden_dim
        
        self.W_xh = nn.Linear(input_dim, hidden_dim)
        self.W_hh = nn.Linear(hidden_dim, hidden_dim, bias=False)
        self.W_tau = nn.Linear(hidden_dim, hidden_dim)
        
    def forward(self, x, h):
        tau = torch.sigmoid(self.W_tau(h)) + 0.1
        dh = -h / tau + torch.tanh(self.W_xh(x) + self.W_hh(h))
        return h + 0.1 * dh
        
class LTC_LSTM(nn.Module):
    def __init__(self, input_dim=22, hidden_dim=64, output_dim=4, num_layers=2):
        super(LTC_LSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        # LTC layers
        self.ltc_layers = nn.ModuleList([
            LTC_Cell(input_dim if i == 0 else hidden_dim, hidden_dim)
            for i in range(self.num_layers)
        ])

        # LSTM branch
        self.lstm = nn.LSTM(
            input_size=hidden_dim,
            hidden_size=hidden_dim,
            num_layers=1,
            batch_first=True
        )

        # Classifier
        self.classifier = nn.Sequential(
            nn.Linear(hidden_dim, 32),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(32, output_dim)
        )

    def forward(self, x):
        batch_size, seq_len, _ = x.size()
        
        # Initialize LTC hidden states
        hiddens = [torch.zeros(batch_size, self.hidden_dim, device=x.device)
                   for _ in range(self.num_layers)]
        
        all_hidden = []
        
        # Process through LTC layers
        for t in range(seq_len):
            x_t = x[:, t, :]
            for layer_idx in range(self.num_layers):
                hiddens[layer_idx] = self.ltc_layers[layer_idx](
                    x_t if layer_idx == 0 else hiddens[layer_idx - 1],
                    hiddens[layer_idx]
                )
            all_hidden.append(hiddens[-1])
        
        # Stack LTC outputs [batch, seq_len, hidden_dim]
        hidden_stack = torch.stack(all_hidden, dim=1)

        # Process through LSTM
        _, (h_n, _) = self.lstm(hidden_stack)
        lstm_context = h_n.squeeze(0)  # [batch, hidden_dim]

        # Final classification
        output = self.classifier(lstm_context)
        
        return output


In [8]:
# Initialize model
import os
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = LTC_LSTM(input_dim=22, hidden_dim=64, output_dim=4, num_layers=2).to(device)
model_path = 'ltc_lstm_model.pth'
if os.path.exists(model_path):
    model.load_state_dict(torch.load(model_path))
    print("Loaded model from checkpoint.")
else:
    print("No checkpoint found. Starting from scratch.")

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5)

num_epochs = 100
previous_val_acc = None
same_acc_streak = 0
best_val_acc = 0
val_acc_list = []

for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    
    for batch, (data, labels) in enumerate(train_loader):
        # Move data to device
        data, labels = data.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, labels)
        loss.backward()
        
        nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        
        train_loss += loss.item()
    
    # Validation
    model.eval()
    val_loss, correct, total = 0, 0, 0
   
    
    with torch.no_grad():
        for data, labels in test_loader:
            data, labels = data.to(device), labels.to(device)
            outputs = model(data)
            val_loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    val_acc = 100 * correct / total
    train_loss /= len(train_loader)
    val_loss /= len(test_loader)
    scheduler.step(val_loss)

    if previous_val_acc is None or val_acc != previous_val_acc:
        same_acc_streak = 0  # Reset counter if there's any change in accuracy
    else:
        same_acc_streak += 1  # Increment counter if accuracy is the same

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), 'ltc_lstm_model.pth')

    val_acc_list.append(val_acc)
    print(f'Epoch {epoch+1}/{num_epochs} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}% | 'f'Same accuracy streak: {same_acc_streak}/10')

    # Early stopping triggered after 10 consecutive epochs with no change in accuracy
    if same_acc_streak >= 10:
        print(f'\nEarly stopping triggered after {epoch+1} epochs!')
        break
    
    # Update previous_val_acc for the next iteration
    previous_val_acc = val_acc

# Load best model and final evaluation
avg_val_acc = sum(val_acc_list) / len(val_acc_list)
print(f'\nTraining complete. Best validation accuracy: {best_val_acc:.2f}%')
print(f'Average validation accuracy over {len(val_acc_list)} epochs: {avg_val_acc:.2f}%')


No checkpoint found. Starting from scratch.
Epoch 1/100 | Train Loss: 1.3890 | Val Loss: 1.3902 | Val Acc: 25.93% | Same accuracy streak: 0/10
Epoch 2/100 | Train Loss: 1.3891 | Val Loss: 1.3924 | Val Acc: 24.07% | Same accuracy streak: 0/10
Epoch 3/100 | Train Loss: 1.3766 | Val Loss: 1.3958 | Val Acc: 24.07% | Same accuracy streak: 1/10
Epoch 4/100 | Train Loss: 1.3798 | Val Loss: 1.4023 | Val Acc: 24.07% | Same accuracy streak: 2/10
Epoch 5/100 | Train Loss: 1.3605 | Val Loss: 1.4097 | Val Acc: 22.22% | Same accuracy streak: 0/10
Epoch 6/100 | Train Loss: 1.3442 | Val Loss: 1.4229 | Val Acc: 24.07% | Same accuracy streak: 0/10
Epoch 7/100 | Train Loss: 1.3311 | Val Loss: 1.4342 | Val Acc: 24.07% | Same accuracy streak: 1/10
Epoch 8/100 | Train Loss: 1.3464 | Val Loss: 1.4355 | Val Acc: 24.07% | Same accuracy streak: 2/10
Epoch 9/100 | Train Loss: 1.3328 | Val Loss: 1.4361 | Val Acc: 24.07% | Same accuracy streak: 3/10
Epoch 10/100 | Train Loss: 1.3373 | Val Loss: 1.4365 | Val Acc: 2